In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1247334/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_11_1_2,0.967668,0.902277,0.975597,0.942702,0.035782,0.138506,0.047508,0.095684,"Hidden Size=[30], regularizer=0.01, learning_r..."
1,model_11_1_1,0.966398,0.921131,0.981956,0.954662,0.037188,0.111784,0.035127,0.075710,"Hidden Size=[30], regularizer=0.01, learning_r..."
2,model_11_1_3,0.957469,0.869773,0.960705,0.919927,0.047069,0.184576,0.076498,0.133716,"Hidden Size=[30], regularizer=0.01, learning_r..."
3,model_11_1_4,0.944739,0.851707,0.918076,0.888422,0.061158,0.210182,0.159488,0.186326,"Hidden Size=[30], regularizer=0.01, learning_r..."
4,model_11_1_0,0.944291,0.958710,0.994495,0.978426,0.061654,0.058522,0.010718,0.036026,"Hidden Size=[30], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
275,model_14_6_13,0.856183,0.624303,0.832799,0.751034,0.159163,0.409410,0.304956,0.360255,"Hidden Size=[35], regularizer=0.1, learning_ra..."
276,model_14_6_10,0.856179,0.634811,0.829719,0.753397,0.159167,0.397958,0.310574,0.356836,"Hidden Size=[35], regularizer=0.1, learning_ra..."
277,model_14_6_9,0.855605,0.633564,0.830079,0.753113,0.159803,0.399318,0.309918,0.357247,"Hidden Size=[35], regularizer=0.1, learning_ra..."
281,model_3_1_0,0.854481,0.824118,0.615951,0.732255,0.161047,0.099760,0.315705,0.201381,"Hidden Size=[10], regularizer=0.01, learning_r..."
